Run these command to download the dataset:
```sh
wget http://files.fast.ai/data/dogscats.zip
unzip dogscats.zip
```

In [1]:
!ls dogscats/

models	sample	test1  train  valid


In [2]:
import tqdm
import torch
import torch.optim as opt
import torch.nn as nn
from torch.utils import data as td
import torchvision as tv
import torchvision.transforms as tr
from torchvision.models import resnet34

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
train_trans = tr.Compose([
    tr.CenterCrop(224),
    tr.RandomHorizontalFlip(0.25),
    tr.RandomHorizontalFlip(0.5),
    tr.ToTensor(), 
    tr.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [5]:
train_iter = tv.datasets.ImageFolder('dogscats/train', transform=train_trans)
train_loader = td.DataLoader(train_iter, batch_size=32, shuffle=True, drop_last=True)

In [6]:
len(train_loader)

718

In [7]:
valid_num_imgs = !find dogscats/valid/ -type f  | wc -l
valid_num_imgs = int(valid_num_imgs[0])
valid_num_imgs

2000

In [8]:
valid_trans = tr.Compose([
    tr.CenterCrop(224), 
    tr.ToTensor(), 
    tr.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [16]:
valid_iter = tv.datasets.ImageFolder('dogscats/valid', transform=valid_trans)
valid_loader = td.DataLoader(valid_iter, batch_size=64, shuffle=False)

In [10]:
net = resnet34(pretrained=True).to(device)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = opt.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [14]:
def accuracy():
    total = 0
    correct= 0
    with torch.no_grad():
        for data in valid_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            _, predicts = torch.max(outputs, 1)
            total += labels.size()[0]
            correct += (predicts == labels).sum().cpu().item()
    return correct/total

In [18]:
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for data in tqdm.tqdm_notebook(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(running_loss, accuracy())


24.360243514180183 0.59



15.75531953573227 0.5805


Left to do:
1. Freeze lower resnet layers
1. ~~Add validation set and accuracy~~
1. Why accuracy is slow? Cache validation set and move to device
1. Improve accuracy!
1. Print loss and accuracy in tdqm
1. Better transformations, take from fast.ai
1. LR scheduling and optimization
1. Different optimizers and hyper-parameters